In [1]:
!pip install --upgrade numpy
!pip install --upgrade scipy

!pip install --upgrade scikit-learn

!pip install --upgrade tensorflow

!pip install --upgrade opencv-python
!pip install --upgrade pandas
!pip install --upgrade tqdm

!pip install --upgrade keras

  Using cached numpy-2.2.3-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
Using cached numpy-2.2.3-cp311-cp311-macosx_14_0_arm64.whl (5.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.3 which is incompatible.
numba 0.57.0 requires numpy<1.25,>=1.21, but you have numpy 2.2.3 which is incompatible.
tensorflow-macos 2.13.0 requires keras<2.14,>=2.13.1, but you have keras 3.8.0 which is incompatible.
tensorflow-macos 2.13.0 requires numpy<=1.24.3,>=1.22, bu

      Successfully uninstalled numpy-2.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
numba 0.57.0 requires numpy<1.25,>=1.21, but you have numpy 2.0.2 which is incompatible.
tensorflow-macos 2.13.0 requires keras<2.14,>=2.13.1, but you have keras 3.8.0 which is incompatible.
tensorflow-macos 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 2.0.2 which is incompatible.
tensorflow-macos 2.13.0 requires tensorboard<2.14,>=2.13, but you have tensorboard 2.18.0 which is incompatible.


In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


print(f"TensorFlow: {tf.__version__}")
print(f"NumPy: {np.__version__}")
print(f"OpenCV: {cv2.__version__}")
print(f"Pandas: {pd.__version__}")

TensorFlow: 2.18.0
NumPy: 2.0.2
OpenCV: 4.11.0
Pandas: 2.2.3


In [4]:
def load_model(base_model):
    return tf.keras.applications.__dict__[base_model](
        include_top=False, weights='imagenet', input_shape=(224, 224, 3), pooling='avg'
    )
models = {
    "DenseNet201": load_model("DenseNet201"),
    "InceptionV3": load_model("InceptionV3"),
    "MobileNetV2": load_model("MobileNetV2"),
}

In [5]:
import numpy as np

def train_hybrid_model(fe_model, classifier_model, X_train, y_train, mask_train=None):
    if mask_train is not None:
        X_train = X_train * mask_train 
    features = fe_model.predict(X_train)
    
    classifier_model.fit(features, y_train)
    
    y_pred = classifier_model.predict(features)
    
    accuracy = accuracy_score(y_train, y_pred)
    precision = precision_score(y_train, y_pred, average='weighted')
    recall = recall_score(y_train, y_pred, average='weighted')
    f1 = f1_score(y_train, y_pred, average='weighted')
    
    return accuracy, precision, recall, f1


In [6]:
for model_name, model in models.items():
    print(f"Basic Summary for {model_name}:")
    num_params = model.count_params()
    num_layers = len(model.layers)
    print(f"Total Parameters: {num_params}")
    print(f"Number of Layers: {num_layers}")
    print("\n")


Basic Summary for DenseNet201:
Total Parameters: 18321984
Number of Layers: 708


Basic Summary for InceptionV3:
Total Parameters: 21802784
Number of Layers: 312


Basic Summary for MobileNetV2:
Total Parameters: 2257984
Number of Layers: 155




In [7]:
def load_ml_model(model_name):
    if model_name == 'SVC':
        return SVC()
    elif model_name == 'KNN':
        return KNeighborsClassifier()
    elif model_name == 'DecisionTree':
        return DecisionTreeClassifier()
    elif model_name == 'MLP':
        return MLPClassifier()
    else:
        raise ValueError(f"Model {model_name} is not defined!")

ml_models = {
    "SVC": load_ml_model("SVC"),
    "KNN": load_ml_model("KNN"),
    "DecisionTree": load_ml_model("DecisionTree"),
    "MLP": load_ml_model("MLP")
}

In [8]:
for model_name, model in ml_models.items():
    print(f"Parameters for {model_name}:")
    print(model.get_params())
    print("-" * 50)


Parameters for SVC:
{'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
--------------------------------------------------
Parameters for KNN:
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
--------------------------------------------------
Parameters for DecisionTree:
{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': None, 'splitter': 'best'}
--------------------------------------------------
Parameters for MLP:
{'activation': 'relu', 'alp

In [9]:
hybrid_architectures = {}

for fe_name, fe_model in models.items():
    for ml_name, ml_model in ml_models.items():
        hybrid_name = f"{fe_name}_{ml_name}"
        hybrid_architectures[hybrid_name] = {
            "Feature Extractor": fe_model,
            "Classifier": ml_model
        }

In [10]:
count = 0
for hybrid_name, hybrid_model in hybrid_architectures.items():
    count += 1
    print(f"Hybrid Architecture {count}: {hybrid_name}")
    print(f"Feature Extractor: {hybrid_model['Feature Extractor']}")
    print(f"Classifier: {hybrid_model['Classifier']}")
    print("\n")

print(f"Total Hybrid Architectures: {count}")


Hybrid Architecture 1: DenseNet201_SVC
Feature Extractor: <Functional name=densenet201, built=True>
Classifier: SVC()


Hybrid Architecture 2: DenseNet201_KNN
Feature Extractor: <Functional name=densenet201, built=True>
Classifier: KNeighborsClassifier()


Hybrid Architecture 3: DenseNet201_DecisionTree
Feature Extractor: <Functional name=densenet201, built=True>
Classifier: DecisionTreeClassifier()


Hybrid Architecture 4: DenseNet201_MLP
Feature Extractor: <Functional name=densenet201, built=True>
Classifier: MLPClassifier()


Hybrid Architecture 5: InceptionV3_SVC
Feature Extractor: <Functional name=inception_v3, built=True>
Classifier: SVC()


Hybrid Architecture 6: InceptionV3_KNN
Feature Extractor: <Functional name=inception_v3, built=True>
Classifier: KNeighborsClassifier()


Hybrid Architecture 7: InceptionV3_DecisionTree
Feature Extractor: <Functional name=inception_v3, built=True>
Classifier: DecisionTreeClassifier()


Hybrid Architecture 8: InceptionV3_MLP
Feature Extractor:

In [11]:
metadata = pd.read_csv("../augmented_images_details.csv")

In [12]:
metadata

,image_path,mask_path,category,label,split
0,../aug-processed-dataset/benign/aug_1.jpg,../aug-processed-dataset/benign/aug_1_mask.png,benign,benign,train
1,../aug-processed-dataset/benign/aug_2.jpg,../aug-processed-dataset/benign/aug_2_mask.png,benign,benign,val
2,../aug-processed-dataset/benign/aug_3.jpg,../aug-processed-dataset/benign/aug_3_mask.png,benign,benign,train
3,../aug-processed-dataset/benign/aug_4.jpg,../aug-processed-dataset/benign/aug_4_mask.png,benign,benign,train
4,../aug-processed-dataset/benign/aug_5.jpg,../aug-processed-dataset/benign/aug_5_mask.png,benign,benign,train
...,...,...,...,...,...
2449,../aug-processed-dataset/malignant/aug_849.jpg,../aug-processed-dataset/malignant/aug_849_mas...,malignant,malignant,val
2450,../aug-processed-dataset/malignant/aug_850.jpg,../aug-processed-dataset/malignant/aug_850_mas...,malignant,malignant,train
2451,../aug-processed-dataset/malignant/aug_851.jpg,../aug-processed-dataset/malignant/aug_851_mas...,malignant,malignant,train
2452,../aug-processed-dataset/malignant/aug_852.jpg,../aug-processed-dataset/malignant/aug_852_mas...,malignant,malignant,val


In [13]:
metadata.describe()

,image_path,mask_path,category,label,split
count,2454,2454,2454,2454,2454
unique,2454,2454,3,3,3
top,../aug-processed-dataset/benign/aug_1.jpg,../aug-processed-dataset/benign/aug_1_mask.png,normal,normal,train
freq,1,1,907,907,1962


In [14]:
split_label_counts = metadata.groupby(['split', 'label']).size().reset_index(name='count')

print(split_label_counts)

   split      label  count
0   test     benign     69
1   test  malignant     85
2   test     normal     90
3  train     benign    555
4  train  malignant    682
5  train     normal    725
6    val     benign     70
7    val  malignant     86
8    val     normal     92


In [15]:
train_data = metadata[metadata['split'] == 'train']
train_data

,image_path,mask_path,category,label,split
0,../aug-processed-dataset/benign/aug_1.jpg,../aug-processed-dataset/benign/aug_1_mask.png,benign,benign,train
2,../aug-processed-dataset/benign/aug_3.jpg,../aug-processed-dataset/benign/aug_3_mask.png,benign,benign,train
3,../aug-processed-dataset/benign/aug_4.jpg,../aug-processed-dataset/benign/aug_4_mask.png,benign,benign,train
4,../aug-processed-dataset/benign/aug_5.jpg,../aug-processed-dataset/benign/aug_5_mask.png,benign,benign,train
5,../aug-processed-dataset/benign/aug_6.jpg,../aug-processed-dataset/benign/aug_6_mask.png,benign,benign,train
...,...,...,...,...,...
2446,../aug-processed-dataset/malignant/aug_846.jpg,../aug-processed-dataset/malignant/aug_846_mas...,malignant,malignant,train
2447,../aug-processed-dataset/malignant/aug_847.jpg,../aug-processed-dataset/malignant/aug_847_mas...,malignant,malignant,train
2450,../aug-processed-dataset/malignant/aug_850.jpg,../aug-processed-dataset/malignant/aug_850_mas...,malignant,malignant,train
2451,../aug-processed-dataset/malignant/aug_851.jpg,../aug-processed-dataset/malignant/aug_851_mas...,malignant,malignant,train


In [16]:
test_data = metadata[metadata['split'] == 'test']
test_data

,image_path,mask_path,category,label,split
9,../aug-processed-dataset/benign/aug_10.jpg,../aug-processed-dataset/benign/aug_10_mask.png,benign,benign,test
35,../aug-processed-dataset/benign/aug_36.jpg,../aug-processed-dataset/benign/aug_36_mask.png,benign,benign,test
42,../aug-processed-dataset/benign/aug_43.jpg,../aug-processed-dataset/benign/aug_43_mask.png,benign,benign,test
51,../aug-processed-dataset/benign/aug_52.jpg,../aug-processed-dataset/benign/aug_52_mask.png,benign,benign,test
68,../aug-processed-dataset/benign/aug_69.jpg,../aug-processed-dataset/benign/aug_69_mask.png,benign,benign,test
...,...,...,...,...,...
2410,../aug-processed-dataset/malignant/aug_810.jpg,../aug-processed-dataset/malignant/aug_810_mas...,malignant,malignant,test
2428,../aug-processed-dataset/malignant/aug_828.jpg,../aug-processed-dataset/malignant/aug_828_mas...,malignant,malignant,test
2437,../aug-processed-dataset/malignant/aug_837.jpg,../aug-processed-dataset/malignant/aug_837_mas...,malignant,malignant,test
2443,../aug-processed-dataset/malignant/aug_843.jpg,../aug-processed-dataset/malignant/aug_843_mas...,malignant,malignant,test


In [17]:
def load_image(image_path, target_size=(224, 224)):
    image = tf.io.read_file(image_path) 
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, target_size)
    image = image / 255.0 
    return image

def load_mask(mask_path, target_size=(224, 224)):
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, target_size) 
    mask = mask / 255.0
    return mask

X_train = np.array([load_image(image_path) for image_path in train_data['image_path']])
mask_train = np.array([load_mask(mask_path) for mask_path in train_data['mask_path']])
y_train = train_data['label'].values

print("Training data shapes:")
print(f"X_train: {X_train.shape}")
print(f"mask_train: {mask_train.shape}")
print(f"y_train: {y_train.shape}")

X_test = np.array([load_image(image_path) for image_path in test_data['image_path']])
mask_test = np.array([load_mask(mask_path) for mask_path in test_data['mask_path']])
y_test = test_data['label'].values

print("\nTesting data shapes:")
print(f"X_test: {X_test.shape}")
print(f"mask_test: {mask_test.shape}")
print(f"y_test: {y_test.shape}")

print("\nData ranges:")
print(f"X_train min: {np.min(X_train)}, max: {np.max(X_train)}")
print(f"mask_train min: {np.min(mask_train)}, max: {np.max(mask_train)}")
print(f"X_test min: {np.min(X_test)}, max: {np.max(X_test)}")
print(f"mask_test min: {np.min(mask_test)}, max: {np.max(mask_test)}")

print("\nClass distribution:")
print("Training labels:", np.unique(y_train, return_counts=True))
print("Testing labels:", np.unique(y_test, return_counts=True))

Training data shapes:
X_train: (1962, 224, 224, 3)
mask_train: (1962, 224, 224, 1)
y_train: (1962,)

Testing data shapes:
X_test: (244, 224, 224, 3)
mask_test: (244, 224, 224, 1)
y_test: (244,)

Data ranges:
X_train min: 0.0, max: 1.0
mask_train min: 0.0, max: 1.0
X_test min: 0.0, max: 1.0
mask_test min: 0.0, max: 1.0

Class distribution:
Training labels: (array(['benign', 'malignant', 'normal'], dtype=object), array([555, 682, 725]))
Testing labels: (array(['benign', 'malignant', 'normal'], dtype=object), array([69, 85, 90]))


In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

ensemble_results = {}

estimator_counts = [3, 5, 7, 9]

for hybrid_name, hybrid_model in hybrid_architectures.items():
    print(f"\nEvaluating Hybrid Architecture: {hybrid_name}")
    
    fe_model = hybrid_model['Feature Extractor']
    ml_model = hybrid_model['Classifier']
    
    X_train_features = fe_model.predict(X_train)
    X_test_features = fe_model.predict(X_test)
    
    hybrid_ensemble_results = {}
    
    for n_estimators in estimator_counts:
        print(f"\nEvaluating Bagging Ensemble with {n_estimators} estimators:")
        
        bagging = BaggingClassifier(
            estimator=ml_model,
            n_estimators=n_estimators,
            max_samples=0.6,
            bootstrap=False,
            random_state=42
        )
        
        bagging.fit(X_train_features, y_train)
        
        ensemble_pred = bagging.predict(X_test_features)
        
        ensemble_accuracy = accuracy_score(y_test, ensemble_pred)
        ensemble_precision = precision_score(y_test, ensemble_pred, average='weighted')
        ensemble_recall = recall_score(y_test, ensemble_pred, average='weighted')
        ensemble_f1 = f1_score(y_test, ensemble_pred, average='weighted')
        
        hybrid_ensemble_results[f'Ensemble_{n_estimators}'] = {
            "Accuracy": ensemble_accuracy,
            "Precision": ensemble_precision,
            "Recall": ensemble_recall,
            "F1 Score": ensemble_f1
        }
        
        print(f"Ensemble with {n_estimators} estimators - "
              f"Accuracy: {ensemble_accuracy:.4f}, Precision: {ensemble_precision:.4f}, "
              f"Recall: {ensemble_recall:.4f}, F1 Score: {ensemble_f1:.4f}")
    
    ensemble_results[hybrid_name] = hybrid_ensemble_results

print("\n\n===== RESULTS SUMMARY =====")
print("Hybrid Architecture | 3 Estimators | 5 Estimators | 7 Estimators | 9 Estimators")
print("-" * 70)

for hybrid_name, results in ensemble_results.items():
    acc_3 = results['Ensemble_3']['Accuracy'] * 100
    acc_5 = results['Ensemble_5']['Accuracy'] * 100
    acc_7 = results['Ensemble_7']['Accuracy'] * 100
    acc_9 = results['Ensemble_9']['Accuracy'] * 100
    
    print(f"{hybrid_name:20} | {acc_3:11.2f}% | {acc_5:11.2f}% | {acc_7:11.2f}% | {acc_9:11.2f}%")



Evaluating Hybrid Architecture: DenseNet201_SVC
62/62 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step

Evaluating Bagging Ensemble with 3 estimators:
Ensemble with 3 estimators - Accuracy: 0.7377, Precision: 0.7361, Recall: 0.7377, F1 Score: 0.7306

Evaluating Bagging Ensemble with 5 estimators:
Ensemble with 5 estimators - Accuracy: 0.7459, Precision: 0.7488, Recall: 0.7459, F1 Score: 0.7382

Evaluating Bagging Ensemble with 7 estimators:
Ensemble with 7 estimators - Accuracy: 0.7459, Precision: 0.7538, Recall: 0.7459, F1 Score: 0.7374

Evaluating Bagging Ensemble with 9 estimators:
Ensemble with 9 estimators - Accuracy: 0.7500, Precision: 0.7565, Recall: 0.7500, F1 Score: 0.7423

Evaluating Hybrid Architecture: DenseNet201_KNN
62/62 ━━━━━━━━━━━━━━━━━━━━ 98s 2s/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step

Evaluating Bagging Ensemble with 3 estimators:
Ensemble with 3 estimators - Accuracy: 0.6762, Precision: 0.6784, Recall: 0.6762, F1 Score: 0.6622

Evaluating Bag

/Users/vishnurchityala/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/vishnurchityala/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Ensemble with 3 estimators - Accuracy: 0.7377, Precision: 0.7352, Recall: 0.7377, F1 Score: 0.7349

Evaluating Bagging Ensemble with 5 estimators:


/Users/vishnurchityala/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/vishnurchityala/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/vishnurchityala/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Ensemble with 5 estimators - Accuracy: 0.7500, Precision: 0.7469, Recall: 0.7500, F1 Score: 0.7462

Evaluating Bagging Ensemble with 7 estimators:


...

Ensemble with 5 estimators - Accuracy: 0.7541, Precision: 0.7526, Recall: 0.7541, F1 Score: 0.7519

Evaluating Bagging Ensemble with 7 estimators:


/Users/vishnurchityala/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/vishnurchityala/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/vishnurchityala/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/vishnurchityala/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.wa

Ensemble with 7 estimators - Accuracy: 0.7582, Precision: 0.7582, Recall: 0.7582, F1 Score: 0.7554

Evaluating Bagging Ensemble with 9 estimators:


/Users/vishnurchityala/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/vishnurchityala/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/vishnurchityala/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/vishnurchityala/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.wa

Ensemble with 9 estimators - Accuracy: 0.7623, Precision: 0.7625, Recall: 0.7623, F1 Score: 0.7598

Evaluating Hybrid Architecture: MobileNetV2_SVC
62/62 ━━━━━━━━━━━━━━━━━━━━ 17s 275ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 268ms/step

Evaluating Bagging Ensemble with 3 estimators:
Ensemble with 3 estimators - Accuracy: 0.7459, Precision: 0.7411, Recall: 0.7459, F1 Score: 0.7370

Evaluating Bagging Ensemble with 5 estimators:
Ensemble with 5 estimators - Accuracy: 0.7623, Precision: 0.7609, Recall: 0.7623, F1 Score: 0.7534

Evaluating Bagging Ensemble with 7 estimators:
Ensemble with 7 estimators - Accuracy: 0.7623, Precision: 0.7606, Recall: 0.7623, F1 Score: 0.7534

Evaluating Bagging Ensemble with 9 estimators:
Ensemble with 9 estimators - Accuracy: 0.7582, Precision: 0.7569, Recall: 0.7582, F1 Score: 0.7496

Evaluating Hybrid Architecture: MobileNetV2_KNN
62/62 ━━━━━━━━━━━━━━━━━━━━ 11s 184ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 238ms/step

Evaluating Bagging Ensemble with 3 estimators:
Ensem

In [25]:
print("\n\n===== RESULTS SUMMARY =====")
print("Hybrid Architecture       | 3 Estimators | 5 Estimators | 7 Estimators | 9 Estimators")
print("-" * 80)

for hybrid_name, results in ensemble_results.items():
    acc_3 = results['Ensemble_3']['Accuracy'] * 100
    acc_5 = results['Ensemble_5']['Accuracy'] * 100
    acc_7 = results['Ensemble_7']['Accuracy'] * 100
    acc_9 = results['Ensemble_9']['Accuracy'] * 100
    
    print(f"{hybrid_name:25} | {acc_3:12.2f}% | {acc_5:12.2f}% | {acc_7:12.2f}% | {acc_9:12.2f}%")



===== RESULTS SUMMARY =====
Hybrid Architecture       | 3 Estimators | 5 Estimators | 7 Estimators | 9 Estimators
--------------------------------------------------------------------------------
DenseNet201_SVC           |        73.77% |        74.59% |        74.59% |        75.00%
DenseNet201_KNN           |        67.62% |        67.21% |        67.21% |        65.98%
DenseNet201_DecisionTree  |        59.84% |        60.25% |        61.07% |        63.93%
DenseNet201_MLP           |        73.77% |        75.00% |        75.82% |        75.82%
InceptionV3_SVC           |        68.85% |        70.08% |        69.26% |        68.44%
InceptionV3_KNN           |        64.34% |        64.34% |        61.89% |        61.48%
InceptionV3_DecisionTree  |        54.51% |        54.10% |        57.38% |        63.11%
InceptionV3_MLP           |        72.95% |        75.41% |        75.82% |        76.23%
MobileNetV2_SVC           |        74.59% |        76.23% |        76.23% |        